In [1]:
import pandas as pd
import numpy as np
import math
import time
import os

import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer

from torchtext import data, datasets
from torchtext.vocab import GloVe

import torch.nn.functional as F

import re

In [2]:
train=pd.read_csv("./data/train.csv")
evaluation=pd.read_csv("./data/evaluation.csv")

In [3]:
#twitter training data
#the data file is available at https://www.kaggle.com/kazanova/sentiment140

twitter_data = pd.read_csv("../sentiment_data_set/training.1600000.processed.noemoticon.csv", encoding = "ISO-8859-1", header=None)
twitter_data.columns = ["sentiment", "time", "date", "query", "username", "text"]
nb_tweets = len(twitter_data["text"])
print(f'{nb_tweets} tweets.')


1600000 tweets.


In [7]:
#Choose a random sample of tweets to put in the train set and another sample for testing accuracy

import random

indices = random.sample(range(nb_tweets), 1600000)

twitter_train_data = twitter_data.iloc[indices[:1000000]]
nb_train_tweets = len(twitter_train_data["text"])
print(f'{nb_train_tweets} train tweets.')

twitter_test_data = twitter_data.iloc[indices[1000000:]]
nb_test_tweets = len(twitter_test_data["text"])
print(f'{nb_test_tweets} test tweets.')


500000 train tweets.
165777 test tweets.


In [8]:
#preprocessing functions

def remove_url(text):
    
    while "http" in text :
        start_url = 0
        while (start_url<len(text)-3 and text[start_url:start_url+4] != "http") :
            start_url += 1
        end_url = start_url + 4
        while (end_url<len(text) and text[end_url] != " ") :
            end_url += 1
            
        text=text[:start_url]+text[end_url+1:]
        
    return text
    

def clean_text(text):
    """
    Applies some pre-processing on the given text.

    Steps :
    - Removing HTML tags
    - Removing punctuation
    - Lowering text
    """
    
    # remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    #remove urls
    text = remove_url(text)
    
    # remove the characters [\], ['] and ["]
    text = re.sub(r"\\", "", text)    
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)    
    
    # convert text to lowercase
    text = text.strip().lower()
    
    # replace punctuation characters with spaces
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)

    return text

In [ ]:
# Transform each text into a vector of word counts
vectorizer = CountVectorizer(stop_words="english",preprocessor=clean_text)

training_features = vectorizer.fit_transform(twitter_train_data["text"])
print("preprocessing train set done")
test_features = vectorizer.transform(twitter_test_data["text"])
print("preprocessing test set done")

# Training
model = LinearSVC()
model.fit(training_features, twitter_train_data["sentiment"])
print("training done")
y_pred = model.predict(test_features)
print("predicting done")

# Evaluation
acc = accuracy_score(twitter_test_data["sentiment"], y_pred)

print("Accuracy on the twitter dataset: {:.2f}".format(acc*100))

preprocessing train set done
preprocessing test set done


In [11]:
#prediction on our covid tweets

covid_train_features = vectorizer.transform(train["text"])
covid_evaluation_features = vectorizer.transform(evaluation["text"])

covid_train_sentiment_pred = model.predict(covid_train_features)
covid_evaluation_sentiment_pred = model.predict(covid_evaluation_features)


In [20]:
train["sentiment"] = covid_train_sentiment_pred
evaluation["sentiment"] = covid_evaluation_sentiment_pred

In [26]:
#writing the new files

np.save("training_v1",train)
np.save("evaluation_v1",evaluation)